In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


def add_layer(inputs, in_size, out_size, n_layer, activation_function = None):
    # 添加神经网络层
    '''
    inputs:              输入数据集
    in_size：            当前层的神经元个数
    out_size:            下一层神经元个数
    n_layer:             第几层
    activation_function: 激活函数
    '''
    layer_name = 'layer%s'% n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):        
            Weights = tf.Variable(tf.truncated_normal([in_size, out_size],stddev=0.1), name='W')
            tf.summary.histogram(layer_name + "/weights", Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + "/biases", biases)
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
            tf.summary.histogram(layer_name + "/outputs", outputs)

        return outputs

In [ ]:


#载入数据集
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

#每个批次的大小
batch_size = 100
#计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

#定义两个placeholder
with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32,[None,784], name='x_input')
    y = tf.placeholder(tf.float32,[None,10], name='y_input')
    # keep_prob用于传入当前层神经元保留的百分比
    keep_prob=tf.placeholder(tf.float32)

#创建一个简单的神经网络
# 隐藏层1
L1 = add_layer(x, 784, 1000, 1, activation_function = tf.nn.tanh)
L1_drop = tf.nn.dropout(L1,keep_prob)
# 隐藏层2
L2 = add_layer(L1_drop, 1000, 500, 2, activation_function = tf.nn.tanh)
L2_drop = tf.nn.dropout(L2,keep_prob) 
# 输出层(即第三层)
prediction = add_layer(L2_drop, 500, 10, 3, activation_function = tf.nn.softmax)


# 二次代价函数（适用于输出神经元为线性函数）
# loss = tf.reduce_mean(tf.square(y-prediction))

with tf.name_scope('loss'):
    #交叉熵代价函数(适用于输出神经元为sigmoid函数和softmax函数)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
    #使用梯度下降法
    train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量
init = tf.global_variables_initializer()

#结果存放在一个布尔型列表中
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))#argmax返回一维张量中最大的值所在的位置
#求准确率
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter(r'./logs/', sess.graph)
    # tensorboard --logdir='logs/'
    sess.run(init)
    
    for epoch in range(31):
        for batch in range(n_batch):
            batch_xs,batch_ys =  mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.7})
        
        result = sess.run(merged, feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        writer.add_summary(result, epoch)
        test_acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
        train_acc = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(test_acc) +",Training Accuracy " + str(train_acc))

In [ ]:
# keep_prob:1.0(没有使用dropout)-----过拟合
Iter 0,Testing Accuracy 0.9338,Training Accuracy 0.933564
Iter 1,Testing Accuracy 0.9453,Training Accuracy 0.949928
Iter 2,Testing Accuracy 0.9524,Training Accuracy 0.960273
Iter 3,Testing Accuracy 0.9582,Training Accuracy 0.966891
Iter 4,Testing Accuracy 0.9605,Training Accuracy 0.971546
Iter 5,Testing Accuracy 0.9626,Training Accuracy 0.975437
Iter 6,Testing Accuracy 0.9632,Training Accuracy 0.977637
Iter 7,Testing Accuracy 0.9662,Training Accuracy 0.981037
Iter 8,Testing Accuracy 0.9676,Training Accuracy 0.982291
...
Iter 28,Testing Accuracy 0.975,Training Accuracy 0.992255
Iter 29,Testing Accuracy 0.9747,Training Accuracy 0.992382
Iter 30,Testing Accuracy 0.9752,Training Accuracy 0.992564

# keep_prob:0.7(使用dropout)-------正常拟合
Iter 0,Testing Accuracy 0.9206,Training Accuracy 0.914273
Iter 1,Testing Accuracy 0.933,Training Accuracy 0.928182
Iter 2,Testing Accuracy 0.9383,Training Accuracy 0.934109
Iter 3,Testing Accuracy 0.9428,Training Accuracy 0.94
Iter 4,Testing Accuracy 0.9457,Training Accuracy 0.946328
Iter 5,Testing Accuracy 0.9478,Training Accuracy 0.9492
Iter 6,Testing Accuracy 0.9502,Training Accuracy 0.953037
Iter 7,Testing Accuracy 0.9518,Training Accuracy 0.954855
Iter 8,Testing Accuracy 0.953,Training Accuracy 0.956728
...
Iter 28,Testing Accuracy 0.9707,Training Accuracy 0.978346
Iter 29,Testing Accuracy 0.9707,Training Accuracy 0.978782
Iter 30,Testing Accuracy 0.9706,Training Accuracy 0.979418